# Plot results scheduling experiments
In this notebook we analyze and  plot the results for scheduling experiments

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from algorithms.MAB import MAB
from algorithms.reward_model import LinearRewardModel, EqualGapRewardModel, CustomRewardModel
from algorithms.bai.ftas import FTaS
from algorithms.bai.tas import TaS
from algorithms.bai.uniform import UniformBAI
from algorithms.fairness_model import ProportionalFairnessModel, VectorFairnessModel, make_prespecified_fairness_model
from tqdm import tqdm
from numpy.typing import NDArray
import pickle
from scipy.stats import bootstrap
import tikzplotlib
import warnings
import seaborn as sns
import matplotlib.ticker as mtick

from scipy.stats._resampling import DegenerateDataWarning

# Filter specific warnings
warnings.filterwarnings("ignore", message="invalid value encountered in scalar divide")
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarning
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered in scalar divide")
warnings.filterwarnings("ignore", category=DegenerateDataWarning, message="The BCa confidence interval cannot be calculated.")

plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
SMALL_SIZE = 10
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

color_blue = '#1a54a6'
color_red = '#1b51g6'
color_green = '#1a54a6'

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def compute_confidence(x):
    mean = np.mean(x)
    ci = bootstrap((x,), statistic=np.mean).confidence_interval
    ci = (np.nan_to_num(ci.low, nan =0), np.nan_to_num(ci.high, nan =0))
    return mean, ci

# Plot allocations

In [ ]:
K = 10
p0 = 0.9
SOLVER = cp.ECOS
NUM_CPUS = 5
DELTA = 0.1
DELTAS = [1e-3, 1e-2, 1e-1]
N_SIM = 10
with open('models/scheduling_env_model/circle_models.pkl', 'rb') as handle:
    THETAs = pickle.load(handle)
models = [
    ('$\\theta$-dependent agonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=True),'Proportional fair agonistic'),
    ('$\\theta$-dependent antagonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=False),'Proportional fair antagonistic'),
]
fig, ax = plt.subplots(1,2*len(models), sharey = True, figsize=(12,2.5))
idx = 0
for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
    instance = MAB(reward_model=reward_model, fairness_model=None)
    [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

    instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
    [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

    ax[idx].plot(np.arange(1,K+1),w, 'k-', label = r"$w^\star$")
    ax[idx].plot(np.arange(1,K+1),w_fair, "g--",label = r"$w_{p}^\star$")
    ax[idx].stem(np.arange(1,K+1), instance.f(instance.THETA), "r--", label = r"$p$")
    ax[idx].grid(linestyle = ":")
    ax[idx].set_xlabel(r'Arms $a$')
    ax[idx].set_title(name)
    ax[idx].set_xlim([1-0.1,K+0.1])
    ax[idx].set_xticks(np.arange(1,K+1))
    
    print(f"[{name}]: T*_{{theta}} = {sol}\nT*_{{theta,p}} = {sol_fair}\n p = {instance.f(instance.THETA)} - psum: {instance.f(instance.THETA).sum()} - pmin: {instance.f(instance.THETA).min()}")
    print('------------------------------------')
    
    idx += 1

ax[0].set_ylabel('Probability')
ax[0].legend()
#plt.suptitle('Proportional fair rates')

models = [
    ('Pre-specified agonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w, p0, agonistic=True), 'Pre-specified model agonistic'),
    ('Pre-specified antagonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w,  p0, agonistic=False), 'Pre-specified model antagonistic'),
]


for idx, (name, reward_model, fairness_model, filename) in enumerate(models):
    instance = MAB(reward_model=reward_model, fairness_model=None)
    [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

    instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
    [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

    ax[2+idx].plot(np.arange(1,K+1),w, 'k-', label = r"$w^\star$")
    ax[2+idx].plot(np.arange(1,K+1),w_fair, "g--",label = r"$w_{p}^\star$")
    ax[2+idx].stem(np.arange(1,K+1), instance.f(instance.THETA), "r--", label = r"$p$")
    ax[2+idx].grid(linestyle = ":")
    ax[2+idx].set_xlabel(r'Arms $a$')
    ax[2+idx].set_title(name)
    ax[2+idx].set_xlim([1-0.1,K+0.1])
    ax[2+idx].set_xticks(np.arange(1,K+1))
    print(f"[{name}]: T*_{{theta}} = {sol}\nT*_{{theta,p}} = {sol_fair}\n p = {instance.f(instance.THETA)} - psum: {instance.f(instance.THETA).sum()} - pmin: {instance.f(instance.THETA).min()}")
    print('------------------------------------')
    idx += 1

plt.savefig("images/scheduling.pdf",dpi = 500,bbox_inches='tight')

# Plot sample complexity results

In [ ]:
box_width = 0.5
points_width = 0.1
alpha_val = 0.3
colors = ["r","g","b"]

algos = list(results.keys())
for DELTA in DELTAS:
    fig, ax = plt.subplots(1,2*len(models), figsize=(16,4))
    models = [
    ('$\\theta$-dependent agonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=True),'Proportional fair agonistic'),
    ('$\\theta$-dependent antagonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=False),'Proportional fair antagonistic'),
    ]
    for i, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ {filename} ------')
        #with open(f'data/{filename}.pkl', 'rb') as f:
        with open('data/scheduler/delta_'+str(DELTA) + '/' + str(filename)+ "_" + str(DELTA) + ".pkl", 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)

        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
            
        #fig, ax = plt.subplots(1, 1)

        for idx, algo_name in enumerate(algos):
            mu_tau, ci_tau = compute_confidence(results[algo_name]['t_vec'])
            mu_perr, ci_err = compute_confidence(results[algo_name]['p_err'])
            print(f"[{algo_name}] Sample complexity: {mu_tau} ({ci_tau}) - P(err): {mu_perr} ({ci_err})")


            ax[i].scatter(idx+1+np.random.rand(len(results[algo_name]['t_vec']))*points_width, results[algo_name]['t_vec'],alpha = alpha_val , c = colors[idx])
            ax[i].boxplot(results[algo_name]['t_vec'],positions = [idx+1],showfliers=False,widths=(box_width), showmeans=True)
        ax[i].set_title(name)
        ax[i].plot(np.arange(6),np.ones(6)*2*sol*np.log(1/DELTA),"b--",label = "LB")
        ax[i].plot(np.arange(6),np.ones(6)*2*sol_fair*np.log(1/DELTA),"r--",label = "LB Fair")
        ax[i].set_xticks([k+1 for k in range(len(results.keys()))], results.keys())
        ax[i].set_xlim([0.5, len(results.keys()) + 0.5])
        ax[i].grid(linestyle = ":")
        #plt.legend()

    models = [
        ('Pre-specified agonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w, p0, agonistic=True), 'Pre-specified model agonistic'),
        ('Pre-specified antagonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w,  p0, agonistic=False), 'Pre-specified model antagonistic'),
    ]

    for i, (name, reward_model, fairness_model, filename) in enumerate(models):
        print(f'------ {filename} ------')
        #with open(f'data/{filename}.pkl', 'rb') as f:
        with open('data/scheduler/delta_'+str(DELTA) + '/' + str(filename)+ "_" + str(DELTA) + ".pkl", 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)
        
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)
        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)
        print(f"[{name}]: T*_{{theta}} = {sol}\nT*_{{theta,p}} = {sol_fair}\n p = {instance.f(instance.THETA)} - psum: {instance.f(instance.THETA).sum()} - pmin: {instance.f(instance.THETA).min()}")
        #fig, ax = plt.subplots(1, 1)
        
        for idx, algo_name in enumerate(algos):
            mu_tau, ci_tau = compute_confidence(results[algo_name]['t_vec'])
            mu_perr, ci_err = compute_confidence(results[algo_name]['p_err'])
            print(f"[{algo_name}] Sample complexity: {mu_tau} ({ci_tau}) - P(err): {mu_perr} ({ci_err})")
            ax[i+2].scatter(idx+1+np.random.rand(len(results[algo_name]['t_vec']))*points_width, results[algo_name]['t_vec'],alpha = alpha_val , c = colors[idx])
            ax[i+2].boxplot(results[algo_name]['t_vec'],positions = [idx+1],showfliers=False,widths=(box_width), showmeans=True)
        ax[i+2].set_title(name)
        ax[i+2].plot(np.arange(6),np.ones(6)*sol*np.log(1/DELTA),"b--",label = "LB")
        ax[i+2].plot(np.arange(6),np.ones(6)*sol_fair*np.log(1/DELTA),"r--",label = "LB Fair")
        ax[i+2].set_xticks([k+1 for k in range(len(results.keys()))], results.keys())
        ax[i+2].set_xlim([0.5, len(results.keys()) + 0.5])
        ax[i+2].grid(linestyle = ":")

        #plt.yscale('log')


    ax[0].legend()
    plt.suptitle('Sample Complexity - $\\delta =$' + str(DELTA))
    plt.savefig(f"images/scheduling_sample_complex_delta_{DELTA}.pdf",dpi = 600,bbox_inches='tight')

# Plot fairness violation results

In [ ]:
for delta in DELTAS:
    idx=0
    fig, ax = plt.subplots(1,4, sharey=True, figsize=(16,4))
    models = [
    ('$\\theta$-dependent agonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=True),'Proportional fair agonistic'),
    ('$\\theta$-dependent antagonistic', CustomRewardModel(THETAs[1]), ProportionalFairnessModel(K, p0=p0, use_gaps=True, invert=False),'Proportional fair antagonistic'),
]

    for _, (name, reward_model, fairness_model, filename) in enumerate(models):
        if 'Equal' in name or 'equal' in name: continue
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/scheduler/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

        instance = MAB(reward_model=reward_model, fairness_model=fairness_model)
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        mask = instance.f(instance.THETA) > 0

        for idxx, algo_name in enumerate(algos):
            w_fair_vec = results[algo_name]['w_vec']

            fairness_res_fbai = np.array(
                [max(0,np.max(instance.f(instance.THETA) - sample))  for x in w_fair_vec  for t, sample in enumerate(x)])

            sns.kdeplot(fairness_res_fbai, label=algo_name, ax=ax[idx], fill=True, alpha=0.3,color = colors[idxx])
        
        ax[idx].set_xlabel(r'Fairness Violation')
        ax[idx].set_ylabel(r'Density' if idx == 0 else '')
        ax[idx].set_title(name)
        ax[idx].grid(linestyle = ":")
        ax[idx].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        ax[0].legend()
        idx+= 1
    plt.suptitle(f'Fairness violations - $\\delta={delta}$')
    #plt.legend()
    models = [
    ('Pre-specified agonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w, p0, agonistic=True), 'Pre-specified model agonistic'),
    ('Pre-specified antagonistic', CustomRewardModel(THETAs[1]), lambda _: make_prespecified_fairness_model(w,  p0, agonistic=False), 'Pre-specified model antagonistic'),
    ]
    for _, (name, reward_model, fairness_model, filename) in enumerate(models):
        if 'Equal' in name or 'equal' in name: continue
        print(f'------ [{delta}]: {filename} ------')
        with open(f'data/scheduler/delta_{delta}/{filename}_{delta}.pkl', 'rb') as f:
            results = pickle.load(f)
        instance = MAB(reward_model=reward_model, fairness_model=None)
        [w,sol,t] = instance.solve_T_star(SOLVER=SOLVER)

        instance = MAB(reward_model=reward_model, fairness_model=fairness_model(w))
        [w_fair,sol_fair,t] = instance.solve_T_star(SOLVER=SOLVER, FAIR=True)

        mask = instance.f(instance.THETA) > 0

        for idxx, algo_name in enumerate(algos):
            w_fair_vec = results[algo_name]['w_vec']

            fairness_res_fbai = np.array(
                [max(0,np.max(instance.f(instance.THETA) - sample))  for x in w_fair_vec  for t, sample in enumerate(x)])

            sns.kdeplot(fairness_res_fbai, label=algo_name, ax=ax[idx], fill=True, alpha=0.3,color = colors[idxx])
        
        ax[idx].set_xlabel(r'Fairness Violation')
        ax[idx].set_ylabel(r'Density' if idx == 0 else '')
        ax[idx].set_title(name)
        ax[idx].grid(linestyle = ":")
        ax[idx].xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
        idx+= 1
        plt.suptitle(f'Fairness violations $\\delta={delta}$')
        
    plt.savefig(f"images/BAI/scheduler/fairness_violations_{delta}.pdf", bbox_inches='tight',dpi = 500)
    plt.show()